# Assignment Class 29

We have a Django project in which we need to **control** a file upload.

Two file **restrictions**:
1) Upload a file in django which should accept only .docx file
1) should not exceed 500kb in size

---

To get started we need to make a **model** with a `FileUpload()` field

In our case, we have *StudentModel* which has a *file* field:

```python

from django.db import models 

class StudentModel(models.Model):
    # Other fields
    ...
    file = models.FileField(null=True, upload_to='files/')

```

And we `makemigrations` and also `migrate`

---

**We need to create a form** that houses this *file* upload. We are going to create a **ModelForm**

In *app_name/forms.py*

```python

from django import forms 
from .models import * 

# Then we create a ModelForm (which creates a form based off of our model)
class StudentForm(forms.ModelForm):
    class Meta:
        model = StudentModel 
        fields = '__all__'

```

Becuase our `fields = '__all__'` it will have a file upload field becuase of our file field in our models.py

Now we need to **render** this form in your template so in our **views.py**

we can go ahead and write *two situtions* one for **POST** and one for **GET** method

```python

def demo_form(request):
    if request.method == "POST":
        # put the form as an object (make sure we import the form from forms.py)
        # we have request.FILES because of form uploads
        student_form = StudentForm(request.POST, request.FILES)
        
        # then since this is a post form, we need to check if the submission if valid 
        if student_form.is_valid():
            # save our form 
            student_form.save() 
            # This is where the url has a namespace of homepage
            return redirect('homepage')
        
    else:
        # GET request will just display the form
        student_form = StudentForm(request.POST, request.FILES)
        
        # creating our context to submit to our template
        context =  {
            'student_form': student_form
        }
        # then we render 
        return render(request, 'demo_form.html', context)

```

In *demo_form.html* we have a **Form tag** a **Cross Reference Forgery Token** and then we display the actual **form** itself with a **submit** button

```html

<html>
    <body>
        <!-- If using file must use enctype -->
        <form method="post" enctype="multipart/form-data">
            {% csrf_token %}
            {{ student_form }}
            <input type='submit' value="submit"/>
        </form>
        
    </body>
</html>

```

---

Now more **focused** on the assignment itself, how do we create these **restrictions** 

Following a [stackoverflow link](https://stackoverflow.com/questions/2472422/django-file-upload-size-limit)

We create a file within the *app/fileChecker.py*

We import `FileField` to make our down `FileField` with some changes to `self.content_types` and `self.max_upload_size`

These two things will be provided when we make our field within `models.py` 

```python

from django.db.models import FileField
from django import forms
from django.template.defaultfilters import filesizeformat

class ContentTypeRestrictedFileField(FileField):
    def __init__(self, *args, **kwargs):
        # Define content types and max upload size from kwargs
        self.content_types = kwargs.pop("content_types", [])
        self.max_upload_size = kwargs.pop("max_upload_size", 0) 
        
        # Supering your child class makes sure it follows MRO 
        super(ContentTypeRestrictedFileField, self).__init__(*args, **kwargs)

```

Then we create our `clean()` function which makes sure our **file** follows our conditions or else it raises an error

```python

def clean(self, *args, **kwargs):
    data = super(ContentTypeRestrictedFileField, self).clean(*args, **kwargs)
    file = data.file

    
    try:
        content_type = file.content_type
        # Makes sure we have have our content type first (doc file)
        if content_type in self.content_types:
            # Makes sure the size isn't more than our max
            if file.size > self.max_upload_size:
                raise forms.ValidationError(
                    'Please keep filesize under %s. Current filesize %s' % 
                    (filesizeformat(self.max_upload_size), filesizeformat(file.size))
                )
        else:
            raise forms.ValidationError('Filetype not supported.')
    except AttributeError:
        # Fallback in case file object does not have expected attributes
        pass

    return data

```

---

**Afterwards** we import this Checker FileField and put this as a replacement for `models.FileField` and make sure we add in `content_types` and `max_upload_size`:

`document = ContentTypeRestrictedFileField(null=True, upload_to='files/', content_types=['application/vnd.openxmlformats-officedocument.wordprocessingml.document', ],max_upload_size=90000)`


There was an **error** in our view function. we need to make sure every action returns a *render*, *redirect*, *HttpResponse* it has to return something

In our case:

```python

def demo_form(request):
    if request.method == 'POST':
        student = StudentForm(request.POST, request.FILES)
        if student.is_valid():
            student.save()
            return redirect('homepage')
        # FIXED CODE
        # --------------------------------------------------------------------------------------
        # if the form isn't valid we'll still render the demo form
        return render(request, 'demo_form.html', {'student':student})
        # --------------------------------------------------------------------------------------
    else:
        # GET request 
        student = StudentForm(request.POST, request.FILES)
        context = {}
        context['student'] = student 
        return render(request, 'demo_form.html', context)

```

We needed to add another render in case the form **fails**
